<a href="https://colab.research.google.com/github/balaksuiuc/CS598IQVIAClaims/blob/main/src/cost_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CS598 Deep Learning for Healthcare**

## **1. Setup**

### 1.1 Change the google colab settings
We can use a GPU on the google colab by setting below.  
**Edit -> Notebook setting -> Hardware accelerator -> GPU**

### 1.2 Check if the GPU is available in the Colab environment

In [ ]:
# The code in this cell is inspired by https://mccormickml.com/2019/07/22/BERT-fine-tuning/
import tensorflow as tf

# Get the GPU device name
device_name = tf.test.gpu_device_name()
print('Device name: {}'.format(device_name))

Device name: 


### 1.3 GPU setting for PyTourch

In [ ]:
# The code in this cell is inspired by https://mccormickml.com/2019/07/22/BERT-fine-tuning/
import torch

# Tell PyTorch to use the GPU
device = torch.device("cuda")
print('GPU:', torch.cuda.get_device_name(0))

RuntimeError: ignored

### 1.4 import necessary packages


In [ ]:
import os
import pickle
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

1.5 Set seed

In [ ]:
# set seed
seed = 100
np.random.seed(seed)
torch.manual_seed(seed)


## **2. Dataset loading**

### 2.1 Load the IQVIA data from google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

[Note]
Need to upload the iqvia data to your google drive

In [ ]:
DATA_DIR = '/content/drive/MyDrive/iqvia_data/'

In [ ]:
import pandas as pd
ENROLL_FILE = DATA_DIR + 'enroll_synth.dat'
CLAIMS_2019 = DATA_DIR + 'claims_2019.dat'
CLAIMS_2018 = DATA_DIR + 'claims_2018.dat'
CLAIMS_2017 = DATA_DIR + 'claims_2017.dat'
CLAIMS_2016 = DATA_DIR + 'claims_2016.dat'
CLAIMS_2015 = DATA_DIR + 'claims_2015.dat'

df_enroll = pd.read_csv(ENROLL_FILE, sep='|', low_memory=False)

df_claims2019 = pd.read_csv(CLAIMS_2019, sep='|', low_memory=False)
df_claims2018 = pd.read_csv(CLAIMS_2018, sep='|', low_memory=False)
df_claims2017 = pd.read_csv(CLAIMS_2017, sep='|', low_memory=False)
df_claims2016 = pd.read_csv(CLAIMS_2016, sep='|', low_memory=False)
df_claims2015 = pd.read_csv(CLAIMS_2015, sep='|', low_memory=False)

## Add year and create a single dataset for claims
df_claims2015["year"] = 2015
df_claims2016["year"] = 2016
df_claims2017["year"] = 2017
df_claims2018["year"] = 2018
df_claims2019["year"] = 2019

list_of_claims = [df_claims2015, df_claims2016, df_claims2017, df_claims2018, df_claims2019]
df_claims = pd.concat(list_of_claims)

In [ ]:
# enroll data
print("Shape of Claims{}".format(df_enroll.shape))
df_enroll.sample(n=5, random_state=0)

In [ ]:
# claim data
print("Shape of Claims{}".format(df_claims.shape))
df_claims.sample(n=5, random_state=0)

### 2.2 Analyze the dataset

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Distribution of patients across regions
rd = df_enroll["pat_region"].value_counts().plot(kind="pie", autopct="%1.1f%%")
rd.set_title("Distribution of patients across regions")

# Distribution of patients gender
rd = df_enroll["der_sex"].value_counts().plot(kind="pie", autopct="%1.1f%%")
rd.set_title("Distribution of patients' gender")

# Distribution of Age
df_enroll["age"] = 2021 - df_enroll["der_yob"]

rd = df_enroll[df_enroll["der_yob"] > 1900]["age"].plot(kind='hist', bins=15)
rd.set_title("Distribution of patients' age")

# Get the count of claims paid (and denied)
df_claims["pmt_st_cd"].value_counts()

# number of diagnosis populated in each claim
diag_cols = ["diag1", "diag2", "diag3", "diag4", "diag5", "diag6", "diag7", "diag8", "diag9", "diag10", "diag11", "diag12"]
df_claims["num_of_diag"] = df_claims[diag_cols].notnull().sum(axis=1)
df_claims["num_of_diag"].mean()

# number of icdprc populated in each claim
icdprc_cols=["icdprc1", "icdprc2", "icdprc3", "icdprc4", "icdprc5", "icdprc6", "icdprc7", "icdprc8", "icdprc9", "icdprc10", "icdprc11", "icdprc12"]
df_claims["num_of_icdprc"] = df_claims[icdprc_cols].notnull().sum(axis=1)
df_claims["num_of_icdprc"].mean()

diag = []
for colname in diag_cols:
    diag.extend(pd.unique(df_claims[colname]))
print(len(np.unique(diag)))
# 22138

prc = []
for colname in icdprc_cols:
    prc.extend(pd.unique(df_claims[colname]))
print(len(np.unique(prc)))
# 926


# number of claims with same day service
sum(df_claims["from_dt"] == df_claims["to_dt"])
# 2378556 out of 2438054 i.e. 97.5%

# Distribution of charges
rd = df_claims["charge"].plot(kind='hist', bins=15)
rd.set_title("Distribution charges")

# Log charges makes more sense
# filtering out rows where charges are less than 1
rd = np.log10(df_claims[df_claims["charge"] > 1]["charge"]).plot(kind='hist', bins=15)
rd.set_title("Distribution of Log of Charges")

# Distribution of Paid amounts
# filtering out rows where paid are less than 1
rd = np.log10(df_claims[df_claims["paid"] > 1]["paid"]).plot(kind='hist', bins=25)
rd.set_title("Distribution of Log of Paid")
plt.show()


# Checking the unique number of patients in the datasets
print(len(pd.unique(df_enroll['pat_id'])))
# 30000
print(len(pd.unique(df_claims2015['pat_id'])))
# 18927
print(len(pd.unique(df_claims2016['pat_id'])))
#21483
print(len(pd.unique(df_claims2017['pat_id'])))
#15190
print(len(pd.unique(df_claims2018['pat_id'])))
#6445
print(len(pd.unique(df_claims2019['pat_id'])))
#4884

## **3. Data embedding**

### 3.1 Embedding

In [ ]:
# Wei's code here
#assign quarter to each record and convert 'to_dt' to datetime format
df_claims["quarter"] = pd.PeriodIndex(pd.to_datetime(df_claims["to_dt"]), freq = 'Q')
df_claims['to_dt'] = pd.to_datetime(df_claims['to_dt'])

#find the length of claims for each patient
df_claims_length = df_claims[['pat_id','to_dt']].groupby('pat_id').agg({'to_dt':['max','min']}).reset_index() 
df_claims_length.columns = ['pat_id','max','min']
df_claims_length['length'] = df_claims_length['max'] - df_claims_length['min']
df_claims_length['length'] = df_claims_length['length'].dt.days
avg_claim_length = df_claims_length['length'].mean()
min_claim_length = df_claims_length['length'].min()
max_claim_length = df_claims_length['length'].max()
print(min_claim_length,avg_claim_length,max_claim_length)
rd = df_claims_length['length'].plot(kind='hist', bins=15)
rd.set_title("Frequency of Length")
#most patients's length is less than three years, 12 quarters, so chooce an observation window of 12 quarters

#calculate index date = last claim - 270 days (180days as prediction window + 90days as last quarter)
claim_indx_date = df_claims[['pat_id','to_dt']].groupby('pat_id').agg({'to_dt':['max']}).reset_index()
claim_indx_date.columns = ['pat_id','max']
claim_indx_date['indx_date'] = claim_indx_date['max'] - pd.to_timedelta(270,unit='d')
#filter claims, 
filterred_claims = pd.merge(df_claims,claim_indx_date,how = 'left',on=['pat_id'])
#observation includes 1000 days(three years) before index date
#prediction window is 180 days (two quarters), last 90 days(last quarter) is reserved as target
filterred_observation = filterred_claims.loc[filterred_claims.to_dt<=filterred_claims.indx_date]
filterred_observation = filterred_observation.loc[filterred_observation.to_dt>=filterred_observation.indx_date-pd.to_timedelta(1095,unit='d')]
filterred_observation = filterred_observation.loc[filterred_observation.to_dt>=pd.to_datetime('2015-10-1')] #code changed after October 1, 2015
filterred_target = filterred_claims.loc[filterred_claims.to_dt>=filterred_claims.indx_date+pd.to_timedelta(180,unit='d')]

#find the number of claims after filter
pat_claimsnumb = filterred_observation[['pat_id']].groupby('pat_id').agg({'pat_id':['count']}).reset_index()
pat_claimsnumb.columns = ['pat_id','count']
print(pat_claimsnumb.shape,pat_claimsnumb['count'].max(),pat_claimsnumb['count'].mean(),pat_claimsnumb['count'].min())
rd = np.log10(pat_claimsnumb['count']).plot(kind='hist', bins=15)
rd.set_title("frequency of claim numbers in log 10")

iag_cols = ["diag1", "diag2", "diag3", "diag4", "diag5", "diag6", "diag7", "diag8", "diag9", "diag10", "diag11", "diag12"]
icdprc_cols=["icdprc1", "icdprc2", "icdprc3", "icdprc4", "icdprc5", "icdprc6", "icdprc7", "icdprc8", "icdprc9", "icdprc10", "icdprc11", "icdprc12"]

#number of unique diag codes and prc codes after filter
diag = []
for colname in diag_cols:
    diag.extend(pd.unique(filterred_observation[colname]))
diag_dict = np.unique(diag)
print(len(np.unique(diag)))

prc = []
for colname in icdprc_cols:
    prc.extend(pd.unique(filterred_observation[colname]))
prc_dict = np.unique(prc)
print(len(np.unique(prc)))

#number of unique record type, procedure code and revenue code(high-level description of services)
filterred_observation["rectype"] = filterred_observation["rectype"].astype('str')
filterred_observation["proc_cde"] = filterred_observation["proc_cde"].astype('str')
filterred_observation["rev_code"] = filterred_observation["rev_code"].astype('str')
print(len(np.unique(filterred_observation["rectype"])))
print(len(np.unique(filterred_observation["proc_cde"])))
print(len(np.unique(filterred_observation["rev_code"])))
rectype_dict = np.unique(filterred_observation["rectype"])
proc_cde_dict = np.unique(filterred_observation["proc_cde"])
rev_code_dict = np.unique(filterred_observation["rev_code"])

#extract interested columns
filtered_features = filterred_observation[["pat_id","paid","charge","quarter","rectype","proc_cde","rev_code","diag1", "diag2", "diag3", 
                                           "diag4", "diag5", "diag6", "diag7", "diag8", "diag9", "diag10", "diag11","diag12","icdprc1", 
                                           "icdprc2", "icdprc3", "icdprc4", "icdprc5", "icdprc6", "icdprc7", "icdprc8", "icdprc9","icdprc10",
                                           "icdprc11", "icdprc12"]]

print(filtered_features.shape)


In [ ]:
#investigate unqiue codes after filter
proc_cde_valuecount = filtered_features["proc_cde"].value_counts()
rev_code_valuecount = filtered_features["rev_code"].value_counts()
diag_valuecount = filtered_features["diag1"].value_counts()
prc_valuecount = filtered_features["icdprc1"].value_counts()

proc_cde_valuecount = proc_cde_valuecount.to_frame()
rd = np.log10(proc_cde_valuecount).plot(kind='hist', bins=15)
rd.set_title("frequency of proc_cde")
#among 6114, less than 500 proc codes appear more than 100 times
rev_code_valuecount = rev_code_valuecount.to_frame()
rd = np.log10(rev_code_valuecount).plot(kind='hist', bins=15)
rd.set_title("frequency of revenue code")
#among 718, less than 300 revenue codes appear more than 100 times
diag_valuecount = diag_valuecount.to_frame()
rd = np.log10(diag_valuecount).plot(kind='hist', bins=15)
rd.set_title("frequency of diag code")
#among 12952, about 1000 diag codes appear more than 100 times
prc_valuecount = prc_valuecount.to_frame()
rd = np.log10(prc_valuecount).plot(kind='hist', bins=15)
rd.set_title("frequency of prc code")
#among 420, less than 50 appear more than 100 times

proc_cde_valuecount = filtered_features["proc_cde"].value_counts().reset_index()
rev_code_valuecount = filtered_features["rev_code"].value_counts().reset_index()
diag_valuecount = filtered_features["diag1"].value_counts().reset_index()
prc_valuecount = filtered_features["icdprc1"].value_counts().reset_index()

#apply thresholds on codes to extract dictionary for high frequency codes
proc_cde_dict = set(proc_cde_valuecount.apply(lambda x: x['index'] if x['proc_cde']>=1000 else 0,axis=1).to_list())
rev_code_dict = set(rev_code_valuecount.apply(lambda x: x['index'] if x['rev_code']>=1000 else 0,axis=1).to_list())
diag_dict = set(diag_valuecount.apply(lambda x: x['index'] if x['diag1']>=1000 else 0,axis=1).to_list())
prc_dict = set(prc_valuecount.apply(lambda x: x['index'] if x['icdprc1']>=100 else 0,axis=1).to_list())
proc_cde_dict.remove(0)
rev_code_dict.remove(0)
diag_dict.remove(0)
prc_dict.remove(0)

print(len(proc_cde_dict),len(rev_code_dict),len(diag_dict),len(prc_dict))
#total code features are reduced to less than 300 after filtering

#filter out code features that not appears in the dictionary
def combine_columns_into_list(x,cols,dict_set):
    out = list()
    for col in cols:
        if x[col] in dict_set:
            out.append(x[col])
    return out

filtered_features['diags'] = filtered_features.apply(lambda x: combine_columns_into_list(x,["diag1","diag2","diag3","diag4","diag5","diag6","diag7","diag8","diag9","diag10"],diag_dict),axis=1)
filtered_features['rev_code'] = filtered_features.apply(lambda x: x['rev_code'] if x['rev_code'] in rev_code_dict else 0,axis=1)
filtered_features['icdprc'] = filtered_features.apply(lambda x: x['icdprc1'] if x['icdprc1'] in prc_dict else 0,axis=1)
filtered_features['proc_cde'] = filtered_features.apply(lambda x: x['proc_cde'] if x['proc_cde'] in proc_cde_dict else 0,axis=1)

#drop original code columns
dropped_features = ["diag1", "diag2", "diag3", "diag4", "diag5", "diag6", "diag7", "diag8", "diag9", "diag10", "diag11", 
                "diag12","icdprc1", "icdprc2", "icdprc3", "icdprc4", "icdprc5", "icdprc6", "icdprc7", "icdprc8", "icdprc9",
                "icdprc10", "icdprc11", "icdprc12"]
filtered_features = filtered_features.drop(dropped_features,axis=1)

print(filtered_features.shape)


In [ ]:
#build multi-hot for rectype, proc_cde, rev_code, recvtype and icdprc
filtered_features = pd.get_dummies(filtered_features,columns = ['rectype','proc_cde','rev_code','icdprc'])
print(filtered_features.shape)
filtered_dict_features = filtered_features.copy(deep=True)

#build multi-hot for diags list feature
v = filtered_dict_features.diags.values
l = [len(x) for x in v.tolist()]
f, u = pd.factorize(np.concatenate(v))
n, m = len(v), u.size
i = np.arange(n).repeat(l)

dummies = pd.DataFrame(
    np.bincount(i * m + f, minlength=n * m).reshape(n, m),
    filtered_dict_features.index, u
)

filtered_dict_features = filtered_dict_features.drop('diags', 1).join(dummies)

#filtered_dict_features.to_pickle('./filterd_feature_dict_281.pkl')
filtered_dict_features = filtered_dict_features.groupby(['pat_id','quarter']).sum().reset_index()
#sort the pat_id and quarter in ascending order
filtered_dict_features = filtered_dict_features.sort_values(by=['pat_id','quarter'],ascending=False)

#maximum quarters of all patients, will truncate to 12
maxnum_quarters = 0
for pat in pat_ids_dict:
    x = filtered_dict_features.loc[filtered_dict_features['pat_id']==pat]
    maxnum_quarters = max(maxnum_quarters,x.shape[0])
print(maxnum_quarters) #13

#construct data for model training
x = np.zeros((18474,279*12))
#build pat_id list to extract data from filtered_dict_features
pat_ids_dict = set(np.unique(filtered_dict_features['pat_id']))
pat_ids_dict = sorted(list(pat_ids_dict),reverse=True)

## construct x matrix
num_pat = 0
for pat in pat_ids_dict:
    pat_val = filtered_dict_features.loc[filtered_dict_features['pat_id']==pat]
    pat_val = pat_val.drop(['pat_id','quarter'],axis=1)
    raw_values = pat_val.values.tolist()
    if len(raw_values)>12:
        raw_values = raw_values[:-1]
    values = [item for sublist in raw_values for item in sublist]
    x[num_pat,:len(values)] = values
    num_pat+=1

#np.savetxt("x_18474_3348.csv", x, delimiter=",")


In [ ]:
#build the y vector
y_paid = filterred_target[['pat_id','paid']].groupby('pat_id').agg({'paid':['sum']}).reset_index() 
y_charge = filterred_target[['pat_id','charge']].groupby('pat_id').agg({'charge':['sum']}).reset_index() 
y_paid.columns = ['pat_id','paid']
y_charge.columns = ['pat_id','charge']
print(y_paid.shape,y_charge.shape)
print(y_paid.isnull().values.any(),y_charge.isnull().values.any())

y_charge['miss'] = y_charge.apply(lambda x: 0 if x['pat_id'] in pat_ids_dict else 1,axis=1)
y_charge = y_charge.loc[y_charge['miss']!=1]
y_charge = y_charge.drop(['miss'],axis=1)
y_charge = y_charge.sort_values(by=['pat_id'],ascending = False)

y_paid['miss'] = y_paid.apply(lambda x: 0 if x['pat_id'] in pat_ids_dict else 1,axis=1)
y_paid = y_paid.loc[y_paid['miss']!=1]
y_paid = y_paid.drop(['miss'],axis=1)
y_paid = y_paid.sort_values(by=['pat_id'],ascending = False)

#patient id is matched row by row between y_paid and x
print(y_charge['pat_id'].to_list()==pat_ids_dict)
print(y_paid['pat_id'].to_list()==pat_ids_dict)
#True True

ypaid = np.array(y_paid['paid'].to_list())
#np.savetxt("ypaid.csv", ypaid, delimiter=",")
ycharge = np.array(y_charge['charge'].to_list())
#np.savetxt("ycharge.csv", ycharge, delimiter=",")


In [ ]:
# Load csv data that is already preprocessed by Wei
# -> Wei's code should be in this ipynb?

In [ ]:
#
def read_emb_csv(filepath):
    pd_x = pd.read_csv(filepath + 'x_18474_3348.csv')
    pd_y_charge = pd.read_csv(filepath + 'ycharge.csv')
    pd_y_paid = pd.read_csv(filepath + 'ypaid.csv')

    return pd_x, pd_y_charge, pd_y_paid

# load the emb csv
EMB_DATA_PATH = '/content/drive/MyDrive/iqvia_data/'
pd_x, pd_y_charge, pd_y_paid = read_emb_csv(EMB_DATA_PATH) 
    

In [ ]:
print("pd_x: {}".format(pd_x.shape))
print("pd_y_charge: {}".format(pd_y_charge.shape))
print("ppd_y_paid: {}".format(pd_y_paid.shape))

In [ ]:
# Input data normalization
# https://betashort-lab.com/%E3%83%87%E3%83%BC%E3%82%BF%E3%82%B5%E3%82%A4%E3%82%A8%E3%83%B3%E3%82%B9/%E6%A9%9F%E6%A2%B0%E5%AD%A6%E7%BF%92/%E3%83%87%E3%83%BC%E3%82%BF%E3%81%AE%E6%AD%A3%E8%A6%8F%E5%8C%96%E3%83%87%E3%83%BC%E3%82%BF%E3%81%AE%E5%89%8D%E5%87%A6%E7%90%86/#toc4

from sklearn.preprocessing import MinMaxScaler
 
scaler_mm = MinMaxScaler()
scaler_mm.fit(pd_x)
pd_x = pd.DataFrame(scaler_mm.transform(pd_x))

In [ ]:
# https://www.coursera.org/learn/cs598-deep-learning-for-healthcare/programming/eJFr3/homework-3-seq2seq/lab
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, x, y):
        if 1:
            # convert to torch.tensor
            self.x = torch.tensor(x.values.astype(np.float32))
            self.y = torch.tensor(y.values.astype(np.float32))
        else:
            self.x = x
            self.y = y
    
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, index):
        #return self.x.loc[index][:], self.y.loc[index]
        return self.x[index][:], self.y[index]

In [ ]:
dataset = CustomDataset(pd_x, pd_y_paid)
print(len(dataset))
#print(dataset[0][0])
#print(dataset[0][0].shape)
#print(dataset[0][1].shape)

### 3.2 Data split

In [ ]:
# https://stackoverflow.com/questions/61811946/train-valid-test-split-for-custom-dataset-using-pytorch-and-torchvision
# https://www.coursera.org/learn/cs598-deep-learning-for-healthcare/programming/eJFr3/homework-3-seq2seq/lab

from torch.utils.data.dataset import random_split

train_len = int(len(dataset)*0.7)
val_len = int(len(dataset)*0.2)
lengths = [train_len, val_len, len(dataset) - train_len - val_len]
train_dataset, val_dataset, test_dataset = random_split(dataset, lengths)

print("train data length: {}".format(len(train_dataset)))
print("val data length: {}".format(len(val_dataset)))
print("test data length: {}".format(len(test_dataset)))

### 3.3 Dataloader

In [ ]:
# https://www.coursera.org/learn/cs598-deep-learning-for-healthcare/programming/eJFr3/homework-3-seq2seq/lab
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(len(train_loader))
print(12931/32)

In [ ]:
# Test
# https://www.coursera.org/learn/cs598-deep-learning-for-healthcare/programming/UEdCb/homework-2-neural-networks/lab

train_iter = iter(train_loader)
x, y = next(train_iter)

print('Shape of a batch x:', x.shape)
print('Shape of a batch y:', y.shape)

## **4. Model building**

### **4.1 Random Forest**

In [ ]:
# Bala, Kautuk

### **4.2 Base model(Drewe-Boss’s paper)**

In [ ]:
# The purpose of this model is to reproduce the prior model proposed by Drewe-Boss et al.  Deep learning for prediction of population health costs

# This code is inspired by 
#  (1) https://www.coursera.org/learn/cs598-deep-learning-for-healthcare/programming/UEdCb/homework-2-neural-networks/lab
#  (2) https://www.coursera.org/learn/cs598-deep-learning-for-healthcare/programming/VNfPA/homework-4-mina/lab

import torch
import torch.nn as nn
import torch.nn.functional as F

class Basemodel(nn.Module):
    def __init__(self, input_size=3348, output_size=1):
        super(Basemodel, self).__init__()
        self.input_size = input_size
        self.hidden_size = 50
        self.output_size = output_size

        self.fc1 = torch.nn.Linear(in_features=self.input_size, out_features=self.hidden_size)
        self.fc2 = torch.nn.Linear(in_features=self.hidden_size, out_features=self.hidden_size)
        self.fc3 = torch.nn.Linear(in_features=self.hidden_size, out_features=self.hidden_size)
        self.fc4 = torch.nn.Linear(in_features=self.hidden_size+self.input_size, out_features=self.output_size)
        self.do = nn.Dropout(0.25)

    def forward(self, x):
        x0 = x
        x = self.do(F.relu(self.fc1(x)))
        x = self.do(F.relu(self.fc2(x)))
        x = self.do(F.relu(self.fc3(x)))
        #print(x.shape)
        #print(x0.shape)
        x = torch.cat((x, x0), dim=1)
        #print(x.shape)
        if 0:
            x = self.do(F.relu(self.fc4(x)))
        else:
            #x = self.fc4(x)
            x = F.relu(self.fc4(x))
        
        return x

model = Basemodel() # Need to specify the input/output size(we have not decided input and output data)
print(model)

### **4.3 Advanced model(Base model + alpha)**

In [ ]:
# Idea

## 5. **Model training**

### 5.1 Loss and potimizer

In [ ]:
import torch.nn as nn

# https://pytorch.org/docs/stable/generated/torch.nn.MSELoss.html
criterion = nn.MSELoss()

# Learning rate is not written in the prior papar. => Experimental value
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

### 5.2 Evaluation

In [ ]:
# https://www.coursera.org/learn/cs598-deep-learning-for-healthcare/programming/UEdCb/homework-2-neural-networks/lab
# https://scikit-learn.org/stable/modules/model_evaluation.html

from sklearn.metrics import *

def regression_metrics(Y_pred, Y_True):
    # Evaluation of methods: 
    # 1. Pearson's correlation (r), 
    # 2. Spearman's correlation (),
    # 3. Mean absolute prediction error (MAPE),
    # 4. R squared (r2),
    # 5. Cumming's Prediction Measure (CPM)
    mape, r2 = mean_absolute_error(Y_True, Y_pred), \
                r2_score(Y_True, Y_pred)

    return mape, r2

def evaluate(model, val_loader):
    model.eval()
    all_y_true = torch.LongTensor()
    all_y_pred = torch.LongTensor()

    val_loss = 0
    for x, y in val_loader:
        y_pred = model(x)
        
        all_y_true = torch.cat((all_y_true, y.to('cpu').long()), dim=0)
        all_y_pred = torch.cat((all_y_pred, y_pred.to('cpu').long()), dim=0)
        loss = criterion(y_pred, y)
        val_loss += loss.item()
    val_loss = val_loss / len(val_loader)
    mape, r2 = regression_metrics(all_y_pred, all_y_true)
    #print(f"mape: {mape:.3f}, r2: {r2:.3f}")
    return val_loss, mape, r2

In [ ]:
# test without training
evaluate(model, train_loader)

### 5.3 Training

In [ ]:
# https://www.coursera.org/learn/cs598-deep-learning-for-healthcare/programming/eJFr3/homework-3-seq2seq/lab

def train(model, train_loader, val_loader, n_epochs):
    model.train()
    for epoch in range(n_epochs):
        train_loss= 0
        for x, y in train_loader:
            optimizer.zero_grad()
            y_pred = model(x)
            #print(y_pred.shape)
            if 1:
                # convert shape from [batch size, 1] to [batch size]
                y_pred = y_pred.view(y_pred.shape[0])
                y = y.view(y.shape[0])
                #print(y_pred.shape)
                #print(y.shape)

            loss = criterion(y_pred, y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_loss = train_loss / len(train_loader)
        val_loss, mape, r2 = evaluate(model, val_loader)
        print('Epoch: {} \t Training Loss: {:.6f} \t validation Loss: {:.6f}'.format(epoch+1, train_loss, val_loss))
        print('Epoch: %d \t Validation mape: %.2f, r2: %.2f'%(epoch+1, mape, r2))


In [ ]:
n_epochs = 10000 # the prior papar's n_epochs=25
train(model, train_loader, val_loader, n_epochs)


### 6. **Test**